# Interesting features
## Query-based features
- Query Performance Prediction
- Query Intent Prediction

- Comparative Query Classification
  - Is there a comparative information need -> comparison intent

- Entity Linking / Query Interpretation
  - Entity count
  - Average entity score / variance of entity score / min / max

## Document-based features
- Genre Classification
- Health Classification
- Readability/Quality/Coherence Features

## Document-query features
- Splade
- monoT5
- BM25

## Expansion -> re-retrieve
- DocT5Query
- LLM Query Expansion
- Entity Linking: BM25 if we use the top entity for retrieval / 0 if no entities

# Data
- `longeval-train-20230513-training`
- `longeval-heldout-20230513-training`
- `longeval-short-july-20230513-training`
- `longeval-long-september-20230513-training`

In [1]:
import pyterrier as pt
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client

ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
def empty_docno(value=1):
    def __transform(df):
        if 'docno' in df.columns:
            return df

        res = df.copy()
        res['docno'] = value

        return res

    return pt.apply.generic(__transform)


def remove_docno():
    return pt.apply.generic(lambda df: df.drop(columns=['docno']))


def columns_to_features(columns, cols_to_keep=('docno', 'query', 'qid')):
    def __transform(df):
        if 'features' in df.columns:
            raise ValueError("Cannot merge features into a dataframe that already has a 'features' column")

        res = df[[col for col in cols_to_keep if col in df.columns]].copy()

        features = df[columns].values
        res['features'] = list(features.reshape((len(features), -1)))

        return res

    return pt.apply.generic(__transform)  #  >> empty_docno()

In [3]:
dataset = pt.get_dataset("irds:ir-benchmarks/longeval-train-20230513-training")
topics = dataset.get_topics(variant='text')

topics

,qid,query
0,q06223196,car shelter
1,q062228,airport
2,q062287,antivirus comparison
3,q06223261,free antivirus
4,q062291,orange antivirus
...,...,...
667,q062224914,tax garden shed
668,q062224961,land of france
669,q062225030,find my training pole job
670,q062225194,gpl car


# Query-based features

In [4]:
topics = dataset.get_topics(variant='text')

qpp = tira.pt.transform_queries('ir-benchmarks/qpptk/all-predictors', dataset) >> columns_to_features(['max-idf', 'avg-idf', 'scq', 'max-scq', 'avg-scq', 'var', 'max-var', 'avg-var', 'wig+10', 'nqc+100', 'smv+100'])
intent_prediction = tira.pt.transform_queries('ir-benchmarks/dossier/pre-retrieval-query-intent', dataset) >> columns_to_features('intent_prediction')

# query_features = empty_docno() >> (intent_prediction ** qpp) >> remove_docno()
query_features = intent_prediction ** qpp

# query_features(topics)

# Document-based features

In [5]:
document_health_classification = tira.pt.transform_documents("ir-benchmarks/fschlatt/document-health-classification", dataset)
genre_mlp_classifier = tira.pt.transform_documents('ir-benchmarks/tu-dresden-01/genre-mlp', dataset)

document_features = document_health_classification ** genre_mlp_classifier

# Document-query features

In [6]:
bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 (tira-ir-starter-pyterrier)', dataset)
monot5 = tira.pt.reranker('ir-benchmarks/tira-ir-starter/MonoT5 Base (tira-ir-starter-gygaggle)', irds_id='ir-benchmarks/longeval-train-20230513-training')

res_bm25 = (bm25 % 10)(topics)
print(res_bm25['docno'].nunique())

res_monot5 = monot5(res_bm25)
print(res_monot5['docno'].nunique())

# doc_query_features = bm25 ** monot5

4808
The run file is normalized outside the TIRA sandbox, I will store it at "/tmp/tmp13gtbm_q-rerank-run".
Done. run file is stored under "/tmp/tmp13gtbm_q-rerank-run/run.txt".


docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.


ValueError: Command ['docker', 'run', '-v', '/root/.tira/pyterrier/ir-benchmarks/longeval-train-20230513-training/irds-cache/:/root/.ir_datasets/:rw', '-v', '/tmp/tmp13gtbm_q-rerank-run:/output/:rw', '--entrypoint', '/irds_cli.sh', 'webis/tira-application:0.0.36', '--output_dataset_path', '/output', '--ir_datasets_id', 'ir-benchmarks/longeval-train-20230513-training', '--rerank', '/output'] did exit with return code 125.

In [ ]:
# full_pipeline = (bm25 % 1000) >> (query_features ** doc_query_features ** document_features)